In [1]:
import nbdev

In [2]:
#| default_exp store

In [3]:
#|export
from __future__ import annotations
from enum import Enum
from types import SimpleNamespace
from typing import List, Callable, TypeVar,  Generic, Sequence, Union, Optional, Any, Set, Tuple, Dict, Protocol, NewType, Literal, TypedDict
from dataclasses import dataclass
from result import Ok, Err, Result #type: ignore

### Svelte Store contract

1. A store must contain a `.subscribe` method, which must accept as its argument a `subscription function`(aka Subscriber or Callback). This `subscription function` must be immediately and synchronously called with the store's current value upon calling `subscribe`. All of a store's active subscription functions must later be synchronously called whenever the store's value changes.

1. The `.subscribe` method must return an `unsubscribe function`(aka Unsubscriber). Calling an `unsubscribe function` must `stop` its subscription, and its corresponding `subscription function` must not be called again by the store.

1. A store may optionally contain a `.set` method, which must accept as its argument a new value for the store, and which synchronously calls all of the store's active subscription functions. Such a store is called a writable store.


~~For interoperability with RxJS Observables, the .subscribe method is also allowed to return an object with an .unsubscribe method, rather than return the unsubscription function directly. Note however that unless .subscribe synchronously calls the subscription (which is not required by the Observable spec), Svelte will see the value of the store as undefined until it does.~~

[Store Contract Documentation](https://svelte.dev/docs#component-format-script-4-prefix-stores-with-$-to-access-their-values-store-contract)

In [4]:
#| export

T = TypeVar("T")
covT = TypeVar("covT", covariant=True)
Subscriber = Callable[[T], None] # a callback
Unsubscriber = Callable[[], None] # a callback to be used upon termination of the subscription
 
class Store(Protocol, Generic[covT]):
    def subscribe(self, subscriber: Subscriber[T]) -> Unsubscriber: ...
    
class Base:  #see SimpleNamespace: https://docs.python.org/3/library/types.html
    def __init__(self, /, **kwargs):
        self.__dict__.update(kwargs)
    def __repr__(self):
        items = (f"{k}={v!r}" for k, v in self.__dict__.items())
        return "{}({})".format(type(self).__name__, ", ".join(items))
    def __eq__(self, other):
        if isinstance(self, Base) and isinstance(other, Base):
           return self.__dict__ == other.__dict__
        return NotImplemented

In [5]:
Updater = Callable[[T], T]

class Readable(Base, Store[T]):
    value: T
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.value!r})"
    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        return lambda: None

class Writable(Readable[T]):
    set: Subscriber
    update: Optional[Callable[[Updater],None]] = None

In [6]:
def writable(initial_value: T) -> Writable[T]:
    value: T = initial_value
    subscribers: Set[Subscriber] = set()

    def subscribe(callback: Subscriber) -> Unsubscriber:
        subscribers.add(callback)
        callback(value)
    
        def unsubscribe() -> None:
            subscribers.remove(callback) if callback in subscribers else None
        return unsubscribe
    
    def _set(new_value: T) -> None:
        nonlocal value
        if new_value != value:
            value = new_value
            for subscriber in subscribers:
                subscriber(value)
    def update(fn: Callable[[T], T]) -> None:
        _set(fn(value))
    return Writable(set=_set, update=update, subscribe=subscribe, value=value, get=lambda: value)

In [7]:
a = writable(1)
u1 = a.subscribe(lambda x: print("1:",x))
a

1: 1


Writable(1)

In [8]:
a.set(2)

1: 2


In [9]:
u1()

In [10]:
a.set(3)

In [11]:
u1()

In [12]:
u1(),u1()

(None, None)

In [13]:
def readable(initial_value: T) -> Readable:
    res = writable(initial_value)
    return Readable(subscribe=res.subscribe, value=res.value, get=lambda: res.value)

In [14]:
b = readable("foo")
b

Readable('foo')

In [15]:
b.subscribe(lambda x: print("2:",x))

2: foo


<function __main__.writable.<locals>.subscribe.<locals>.unsubscribe() -> 'None'>

In [16]:
try:
    b.set("bar")
except Exception as error:
  print(error)


'Readable' object has no attribute 'set'


A `store` that does not change is not useful. A `Readable` is like a writable where there is only one "thing" that can change its value. Lets change writable to add this "thing", which we will call a `Notifier`.

In [17]:
Notifier = Callable[[Subscriber], Union[Unsubscriber, None]]

In [18]:
def writable(initial_value: T, start: Notifier=lambda x: None) -> Writable[T]:
    value: T = initial_value
    stop: Optional[Unsubscriber] = None
    subscribers: Set[Subscriber] = set()

    def subscribe(callback: Subscriber) -> Unsubscriber:
        subscribers.add(callback)
        if (len(subscribers) == 1):
            stop = start(callback) or (lambda: None)
        callback(value)
    
        def unsubscribe() -> None:
            subscribers.remove(callback) if callback in subscribers else None
            if (len(subscribers) == 0):
                nonlocal stop
                stop() if stop else None
                stop = None
        return unsubscribe
    
    def _set(new_value: T) -> None:
        nonlocal value
        if new_value == value: return None
        if not stop: # store is not ready yet
            return None
        value = new_value
        for subscriber in subscribers:
                subscriber(value)
        
    def update(fn: Callable[[T], T]) -> None:
        _set(fn(value))
    return Writable(start=start, set=_set, update=update, subscribe=subscribe, value=value, get=lambda: value)

In [19]:
a = writable(1)
u1 = a.subscribe(lambda x: print("1:",x))
a

1: 1


Writable(1)

In [20]:
u1(), u1()

(None, None)

In [21]:
from threading import Event, Thread

def every(interval, func, *args):
    stopped = Event()
    def loop():
        while not stopped.wait(interval): # the first call is in `interval` secs
            func(*args)
    Thread(target=loop).start()    
    return stopped.set

In [22]:
def start(set):
    count = 0
    def incrementCounter():
        nonlocal count
        count = count +1
        set(count)
    cancel = every(1, incrementCounter)
    return cancel

In [23]:
def myset(x):
    value = x
    print("myset:", value)

In [24]:
stop = start(myset)

In [25]:
import time

time.sleep(3)
stop()

myset: 1
myset: 2
myset: 3


In [26]:
b = writable(0, start)

In [27]:
b

Writable(0)

In [28]:
u1 = b.subscribe(lambda x: print("1:",x))

1: 0


1: 1
1: 2
1: 3
1: 4
1: 5
1: 6


In [29]:
time.sleep(4)
u1()

In [30]:
class Readable(Base, Store[T]):
    value: T
    start: Notifier
    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.value!r})"
    def subscribe(self, callback: Subscriber) -> Unsubscriber:
        return lambda: None
def readable(initial_value: T) -> Readable[T]:
    res = writable(initial_value)
    return Readable(start=start, subscribe=res.subscribe, value=res.value, get=lambda: res.value)

In [ ]:
t.cancel()

In [ ]:
class Start(Protocol):
    def __call__(self, set: Subscriber) -> Unsubscriber: ...

In [ ]:
def start(set):
    
export const time = readable(new Date(), function start(set) {
	const interval = setInterval(() => {
		set(new Date());
	}, 1000);

	return function stop() {
		clearInterval(interval);
	};
});


In [ ]:
def readable(value: Optional[T], start: Optional[TNotifier]=None) -> TReadable:
    """Readable Store factory"""
    return {"subscribe": writable(value, start)['subscribe']}

In [ ]:
from functools import wraps

#https://gist.github.com/JulienPalard/021f1c7332507d6a494b

def curry(func):
    """
    >>> @curry
    ... def foo(a, b, c):
    ...     return a + b + c
    >>> foo(1)
    <function __main__.foo>
    """
    @wraps(func)
    def curried(*args, **kwargs):
        if len(args) + len(kwargs) >= func.__code__.co_argcount:
            return func(*args, **kwargs)

        @wraps(func)
        def new_curried(*args2, **kwargs2):
            return curried(*(args + args2), **dict(kwargs, **kwargs2))

        return new_curried

    return curried

In [ ]:
def writable(init) {
	let _val = init;
	const subs = [];

	const subscribe = (cb) => {
		subs.push(cb);
		cb(_val);

		return () => {
			const index = subs.findIndex((fn) => fn === cb);
			subs.splice(index, 1);
		};
	};

	const set = (v) => {
		_val = v;
		subs.forEach((fn) => fn(_val));
	};

	const update = (fn) => set(fn(_val));

	return { subscribe, set, update };
}

In [ ]:
#| export

T = TypeVar("T")
TEvent = TypeVar('TEvent', bound=object) 
THandler = Callable[[TEvent], None] 
TSubscriber = Union[Callable[[T], None], Callable[[T], Result[Ok[T], Err[TEvent]]]]  # action to invoke when the store changes
TUnsubscriber = Callable[[], None] # action to invoke upon graceful termination of the subscription
# TUpdater = Callable[[T, TEvent], T] # action to invoke to update the store, aka reducer
TUpdater = Callable[[T], T]
noop = lambda *x: None
TNotifier = Callable[[TSubscriber],TUnsubscriber]
class TObserver(Protocol):
    def __call__(self, msg: TEvent) -> None: ... # type: ignore
class TObservable(Protocol):
    def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber: ...
class TSubject(TObservable, TObserver, Protocol): ...
class TActions(TypedDict, total=False):
    START: Optional[Callable]
    UPDATE: TSubscriber
    STOP: Optional[Callable]



In [ ]:
class Base(Generic[T]):
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
        
    def __eq__(self, other):
        return self._value == other._value # you might want to add some type checking here
        
    def __repr__(self) -> str: return f"{self.__class__.__name__}({str(self._value)})"

class Readable(Base[T], TObservable):
    def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber:
        def unsubscribe()-> None: pass
        return unsubscribe
class Writable(Readable, TObservable): 
    def set(self, new_value: T) -> None: pass
    def update(self, fn: TUpdater) -> None: pass

In [ ]:
def writable(value: Optional[T], start: Optional[TNotifier]=None) -> Writable:
    """Writable Store factory"""
    stop: Optional[TUnsubscriber] = None
    subscribers: Set[TSubscriber] = set()
    def _set(new_value: T) -> None:
        nonlocal value
        if (new_value != value) and (stop):
            value = new_value
            for subscriber in subscribers:
                subscriber(value)
    def update(fn: TUpdater) -> None:
        _set(fn(value))
    def subscribe(subscriber: TSubscriber) -> TUnsubscriber:
        subscribers.add(subscriber)
        if (len(subscribers) == 1) and (start is not None): stop = start(_set)
        subscriber(value)
        def unsubscribe()-> None:
            subscribers.remove(subscriber) if subscriber in subscribers else None
            nonlocal stop
            if (len(subscribers) == 0) and (stop is not None): 
                stop() # type: ignore
                stop = None
        return unsubscribe
    return  Writable(set=_set, update= update, subscribe= subscribe, _value=value)

In [ ]:

count = writable(1)

In [ ]:
count.set

In [ ]:
count.subscribe(lambda x: print(x))

In [ ]:
count

In [ ]:
def readable(value: Optional[T], start: Optional[TNotifier]=None) -> TReadable:
    """Readable Store factory"""
    return {"subscribe": writable(value, start)['subscribe']}

In [ ]:
#| export

T = TypeVar("T")

TSubscriber = Callable[[T], None]
TUnsubscriber = Callable[[], None]
TUpdater = Callable[[T], T]
TStartStop = Callable[[TSubscriber], Union[TUnsubscriber, None]]

    
class Readable(Generic[T]):
    def __init__(self, value: T, start: Optional[TStartStop]=None) -> None:
        self.value: T = value
        self.start: Optional[TStartStop] = start
        self.subscribers: set[TSubscriber] = set() # callback list

    def __getattr__(self, name):
        return getattr(self.value, name)

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.value!r})"

    def __str__(self) -> str: return str(self.value)

    def get(self) -> T: return self.value

    def _set(self, new_value: T) -> None:
        if new_value != self.value: self.value = new_value
        for subscriber in self.subscribers: subscriber(self.value)

    def subscribe(self, subscriber: TSubscriber) -> TUnsubscriber:
        self.subscribers.add(subscriber)
        subscriber(self.value)
        if (len(self.subscribers) == 1):
             if self.start is not None: 
                self.stop: Union[TUnsubscriber, None] =  self.start(self._set) 
        def unsubscribe()-> None:
            self.subscribers.remove(subscriber)
        if isinstance(subscriber, Subscriber): subscriber.add_subscription(unsubscribe)
        return unsubscribe

class Writable(Readable[T]):

    def set(self, new_value: T) -> None:
        self._set(new_value)
    
    def update(self, fn: TUpdater) -> None: self.set(fn(self.value))


class Subscriber(Generic[T]):
    """ Represents a subscriber (a callback) to a store (an observable)."""
    def __init__(self, fn: Callable[[T], None], observable: Optional[Union[Readable[T], Writable[T]]]) -> None:
        self.fn = fn
        self.subscriptions: Set[Callable[[], None]] = set()

    def __call__(self, value: T) -> None:
        self.fn(value)

    def __eq__(self, other: Callable[[T], None]) -> bool:
        return self.fn == other.fn

    def __hash__(self) -> int:
        return hash(self.fn)
    
    def __del__(self)-> None:
        for unsubscribe in self.subscriptions: unsubscribe()
        
    def add_subscription(self, unsubscribe: Callable[[], None])-> None:
        self.subscriptions.add(unsubscribe)
    


In [ ]:
a = set()
a.add(1), a.add('foo')
a, a[0]

In [ ]:
a.add(2)
a.add(object())
a.add(1.0)

In [ ]:
[x for x in a]

In [ ]:
Stores = Union[Readable[T], Sequence[Readable[T]]]
Observable = Union[Readable[T], Writable[T]]

In [ ]:
def derived (stores: Stores. fn: Callable) -> Readable:
"""Derived value store by synchronizing one or more readable stores and
applying an aggregation function over its input values."""

In [ ]:
export function writable<T>(value?: T, start: StartStopNotifier<T> = noop): Writable<T> {
	let stop: Unsubscriber;
	const subscribers: Set<SubscribeInvalidateTuple<T>> = new Set();

	function set(new_value: T): void {
		if (safe_not_equal(value, new_value)) {
			value = new_value;
			if (stop) { // store is ready
				const run_queue = !subscriber_queue.length;
				for (const subscriber of subscribers) {
					subscriber[1]();
					subscriber_queue.push(subscriber, value);
				}
				if (run_queue) {
					for (let i = 0; i < subscriber_queue.length; i += 2) {
						subscriber_queue[i][0](subscriber_queue[i + 1]);
					}
					subscriber_queue.length = 0;
				}
			}
		}
	}

	function update(fn: Updater<T>): void {
		set(fn(value));
	}

	function subscribe(run: Subscriber<T>, invalidate: Invalidator<T> = noop): Unsubscriber {
		const subscriber: SubscribeInvalidateTuple<T> = [run, invalidate];
		subscribers.add(subscriber);
		if (subscribers.size === 1) {
			stop = start(set) || noop;
		}
		run(value);

		return () => {
			subscribers.delete(subscriber);
			if (subscribers.size === 0) {
				stop();
				stop = null;
			}
		};
	}

	return { set, update, subscribe };
}

In [ ]:
len(one)


In [ ]:


class Derived:
    def __init__(self, stores: Stores, fn: Callable[[Any], Any], initial_value: Any):
        self.stores = stores
        self.fn = fn
        self.initial_value = initial_value
        self.value = initial_value
        self.subscribers: List[Callable[[Any], None]] = []
        self._unsubscribe: List[Callable[[], None]] = []
        self._set = self.set
        self.set = self.update

    def set(self, value: Any) -> None:
        self.value = value
        for subscriber in self.subscribers: subscriber(self.value)

    def update(self, fn: Callable[[Any], Any]) -> None:
        self.set(fn(self.value))

    def subscribe(self, subscriber: Callable[[Any], None]) -> Callable[[], None]:
        self.subscribers.append(subscriber)
        subscriber(self.value)
        return lambda: self.subscribers.remove(subscriber)

    def unsubscribe(self) -> None:
        for unsub in self._unsubscribe: unsub()

    def __enter__(self) -> Derived:
        self._unsubscribe = [store.subscribe(self._set) for store in self.stores]
        return self

    def __exit__(self, exc_type, exc_value, traceback) -> None:
        self.unsubscribe()

    def __call__(self) -> Any:
        return self.value

    def __repr__(self) -> str:
        return f"{self.__class__.__name__}({self.stores!r}, {self.fn!r}, {self.initial_value!r})"

    def __str__(self) -> str: return str(self.value)

export function derived<S extends Stores, T>(
	stores: S,
	fn: (values: StoresValues<S>, set: (value: T) => void) => Unsubscriber | void,
	initial_value?: T
): Readable<T>;

/**
 * Derived value store by synchronizing one or more readable stores and
 * applying an aggregation function over its input values.
 *
 * @param stores - input stores
 * @param fn - function callback that aggregates the values
 * @param initial_value - initial value
 */
export function derived<S extends Stores, T>(
	stores: S,
	fn: (values: StoresValues<S>) => T,
	initial_value?: T
): Readable<T>;

/**
 * Derived value store by synchronizing one or more readable stores and
 * applying an aggregation function over its input values.
 *
 * @param stores - input stores
 * @param fn - function callback that aggregates the values
 */
export function derived<S extends Stores, T>(
	stores: S,
	fn: (values: StoresValues<S>) => T
): Readable<T>;

export function derived<T>(stores: Stores, fn: Function, initial_value?: T): Readable<T> {
	const single = !Array.isArray(stores);
	const stores_array: Array<Readable<any>> = single
		? [stores as Readable<any>]
		: stores as Array<Readable<any>>;

	const auto = fn.length < 2;

	return readable(initial_value, (set) => {
		let inited = false;
		const values = [];

		let pending = 0;
		let cleanup = noop;

		const sync = () => {
			if (pending) {
				return;
			}
			cleanup();
			const result = fn(single ? values[0] : values, set);
			if (auto) {
				set(result as T);
			} else {
				cleanup = is_function(result) ? result as Unsubscriber : noop;
			}
		};

		const unsubscribers = stores_array.map((store, i) => subscribe(
			store,
			(value) => {
				values[i] = value;
				pending &= ~(1 << i);
				if (inited) {
					sync();
				}
			},
			() => {
				pending |= (1 << i);
			})
		);

		inited = true;
		sync();

		return function stop() {
			run_all(unsubscribers);
			cleanup();
		};
	});
}

/**
 * Get the current value from a store by subscribing and immediately unsubscribing.
 * @param store readable
 */
export { get_store_value as get };

### Export

In [ ]:
nbdev.nbdev_export()

In [ ]:
# #| export

# def writable(value: T) -> Writable[T]:
#     """ Create a writable store with a given value that allows both updating and reading by subscription."""

    
#     def set(new_value: T) -> None:
#         nonlocal value
#         if new_value != value: value = new_value
#         for subscriber in subscriber_queue: subscriber(value)

#     def update(fn: Updater[T]) -> None: set(fn(value))

#     def subscribe(subscriber: Subscriber[T]) -> Unsubscriber:
#         subscriber_queue.append(subscriber)
#         subscriber(value)

#         def unsubscribe() -> None:
#             subscriber_queue.remove(subscriber)

#         return unsubscribe

#     ret = Writable() # type: ignore
#     ret.set = set # type: ignore
#     ret.update = update # type: ignore
#     ret.subscribe = subscribe

#     return ret

In [ ]:
# #| export

# def readable(value: T) -> Readable[T]:
#     ret = Readable()
#     ret.subscribe = writable(value).subscribe
#     return ret

In [ ]:
# class Store:
#     def __init__(self, value):
#         self.value = value
#         self.subscribers = []
#     def subscribe(self, subscriber):
#         self.subscribers.append(subscriber)
#         subscriber(self.value)
#         return lambda: self.subscribers.remove(subscriber)
#     def set(self, value):
#         if value != self.value:
#             self.value = value
#             for subscriber in self.subscribers:
#                 subscriber(value)
#     def update(self, updater):
#         self.set(updater(self.value))